In [16]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
import numpy as np
import matplotlib.pyplot as plt
import os
import gdown
from zipfile import ZipFile
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
os.makedirs("celeba_gan")

url = "https://drive.google.com/uc?id=1O7m1010EJjLE5QxLZiM9Fpjs7Oj6e684"
output = "celeba_gan/data.zip"
gdown.download(url, output, quiet=True)

with ZipFile("celeba_gan/data.zip", "r") as zipobj:
    zipobj.extractall("celeba_gan")

In [5]:
dataset = keras.utils.image_dataset_from_directory(
    "celeba_gan", label_mode=None, image_size=(64, 64), batch_size=32
)
dataset = dataset.map(lambda x: x / 255.0)

Found 202599 files belonging to 1 classes.


In [7]:
discriminator = keras.Sequential([
    keras.Input(shape=(64,64,3)),
    layers.Conv2D(64, kernel_size=4, strides = 2, padding='same'),
    layers.LeakyReLU(0.2),
    layers.Conv2D(128, kernel_size=4, strides = 2, padding='same'),
    layers.LeakyReLU(0.2),
    layers.Conv2D(128, kernel_size=4, strides = 2, padding='same'),
    layers.LeakyReLU(0.2),
    layers.Flatten(),
    layers.Dropout(0.2),
    layers.Dense(1, activation='sigmoid')
])

In [8]:
print(discriminator.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 64)        3136      
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 32, 32, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 128)       131200    
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 16, 16, 128)       0         
                                                                 
 conv2d_2 (Conv2D)           (None, 8, 8, 128)         262272    
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 8, 8, 128)         0         
                                                                 
 flatten (Flatten)           (None, 8192)              0

In [9]:
latent_dim = 128

In [10]:
generator = keras.Sequential([
    keras.Input(shape=(latent_dim,)),
    layers.Dense(8*8*128),
    layers.Reshape((8,8,128)),
    layers.Conv2DTranspose(128, kernel_size=4, strides = 2, padding='same'),
    layers.LeakyReLU(0.2),
    layers.Conv2DTranspose(256, kernel_size=4, strides = 2, padding='same'),
    layers.LeakyReLU(0.2),
    layers.Conv2DTranspose(512, kernel_size=4, strides = 2, padding='same'),
    layers.LeakyReLU(0.2),
    layers.Conv2D(3, kernel_size=5, padding='same', activation='sigmoid'),
])

In [11]:
print(generator.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 8192)              1056768   
                                                                 
 reshape (Reshape)           (None, 8, 8, 128)         0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 16, 16, 128)      262272    
 nspose)                                                         
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 16, 16, 128)       0         
                                                                 
 conv2d_transpose_1 (Conv2DT  (None, 32, 32, 256)      524544    
 ranspose)                                                       
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 32, 32, 256)      

In [12]:
opt_gen = keras.optimizers.Adam(learning_rate = 1e-4)
opt_disc = keras.optimizers.Adam(learning_rate = 1e-4)

In [13]:
loss_fn = keras.losses.BinaryCrossentropy()

In [14]:
epochs = 15

In [18]:
for epoch in range(epochs):
    for idx, real in enumerate(tqdm(dataset)):
        batch_size = real.shape[0]
        random_latent_vectors = tf.random.normal(shape=(batch_size, latent_dim))

       
        fake = generator(random_latent_vectors)

        ## train the discriminator: max y*log(D(x)) + (1-y)*(log (1-D(G(z)))
        with tf.GradientTape() as disc_tape:
            loss_disc_real = loss_fn(tf.ones((batch_size,1)),discriminator(real))
            loss_disc_fake = loss_fn(tf.zeros((batch_size, 1)), discriminator(fake))

            loss_disc = (loss_disc_real + loss_disc_fake)/2

        grads = disc_tape.gradient(loss_disc, discriminator.trainable_weights)
        opt_disc.apply_gradients(
            zip(grads, discriminator.trainable_weights)
        )

        #train generator min log(1 - D(G(z))) <-> max log(D(G(z)))

        with tf.GradientTape() as gen_tape:
            fake = generator(random_latent_vectors)
            output = discriminator(fake)
            loss_gen = loss_fn(tf.ones((batch_size, 1)), output)

        grads = gen_tape.gradient(loss_gen, generator.trainable_weights)
        opt_gen.apply_gradients(
            zip(grads, generator.trainable_weights)
        )


  7%|▋         | 440/6332 [04:49<1:04:35,  1.52it/s]


KeyboardInterrupt: 